In [ ]:
PORTAL_PORT=49051
PORTAL_HOST='0.0.0.0'
DATABASE_PORT=27017


from bson.objectid import ObjectId
import random
import string
import hmac
import hashlib
from hashlib import sha256
from flask import Flask
from flask import request,redirect
from flask import Response
from flask import render_template
from os import listdir
from os.path import isfile, join
import os
from pymongo import MongoClient
import os.path

if os.path.isfile('.ADMIN_PASSWORD'):
    with open('.ADMIN_PASSWORD','r') as f:
        ADMIN_PASSWORD = f.read()
else:
    with open('.ADMIN_PASSWORD','w') as f:
        ADMIN_PASSWORD = "ADMIN_PASSWORD"
        f.write(ADMIN_PASSWORD)


app = Flask(__name__)
app.config['TEMPLATES_AUTO_RELOAD'] = True

@app.after_request
def add_header(r):
    """
    Add headers to both force latest IE rendering engine or Chrome Frame,
    and also to cache the rendered page for 10 minutes.
    """
    r.headers["Cache-Control"] = "no-cache, no-store, must-revalidate"
    r.headers["Pragma"] = "no-cache"
    r.headers["Expires"] = "0"
    r.headers['Cache-Control'] = 'public, max-age=0'
    return r

@app.route('/create')
def create():
    return render_template("formCreate.html")

@app.route('/createError')
def createError():
    return render_template("createError.html")

@app.route('/saveError')
def saveError():
    return render_template("saveError.html")

@app.route("/createSave",methods=['POST'])
def createSave():
    if request.form.get('name', "")=="":
        return redirect("/createError")
    if len(request.form.get('appPassword', ""))<10:
        return redirect("/createError")
    form_text=["name","desc"]
    form_checks=["spisok","read","write","delete"]
    appObject={i:request.form.get(i, "") for i in form_text}
    for i in form_checks:
        t=request.form.get(i, "off")
        if t=="on":
            t="zapros"
        appObject[i]=t
    appObject["salt"]=''.join(random.choices(string.ascii_letters + string.digits, k=10))
    appObject["passwordHash"]=sha256((appObject["salt"]+request.form.get('appPassword', "")).encode('utf-8')).hexdigest()
    appObject["secretKey"]=sha256(''.join(random.choices(string.ascii_letters + string.digits, k=10)).encode('utf-8')).hexdigest()
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    result=db.apps.insert_one(appObject)
    link=db.links.insert_one({"appID":result.inserted_id})
    return redirect(f"/secretKey?link_id={link.inserted_id}")

@app.route("/getApp",methods=['GET'])
def getApp():
    appID=request.args.get('id',"Ошибка")
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    appObject=db.apps.find_one({"_id":ObjectId(appID)})
    return render_template("getApp.html",appObject=appObject)

@app.route("/",methods=['GET'])
def appList():
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    apps=list()
    for i in db.apps.find({}):
        t=i
        m=["spisok","read","write","delete"]
        status="off"
        for i2 in m:
            if i[i2] == "on":
                status="on"
        for i2 in m:
            if i[i2] == "zapros":
                status="zapros"
        t["status"]=status
        apps.append(t)
    return render_template("appList.html",apps=apps,mode="read")
    
@app.route("/admin",methods=['GET'])
def admin():
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    apps=list()
    for i in db.apps.find({}):
        t=i
        m=["spisok","read","write","delete"]
        status="off"
        for i2 in m:
            if i[i2] == "on":
                status="on"
        for i2 in m:
            if i[i2] == "zapros":
                status="zapros"
        t["status"]=status
        apps.append(t)
    return render_template("appList.html",apps=apps,mode="admin")

@app.route("/adminApp")
def adminApp():
    appID=request.args.get('id',"Ошибка")
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    appObject=db.apps.find_one({"_id":ObjectId(appID)})
    return render_template("adminApp.html",appObject=appObject)

@app.route("/adminSave",methods=['POST'])
def adminSave():
    print(request.form)
    if request.form.get('name', "")=="":
        return redirect("/saveError")
    if request.form.get("adminPassword","") != ADMIN_PASSWORD:
        return redirect("/saveError")
    appID=request.form.get('appID', "")
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    db.apps.update_one({"_id":ObjectId(appID)},
                       {'$set': 
                       {
                           "name":request.form.get('name', ""),
                           "decs":request.form.get('appDesc', ""),
                           "spisok":request.form.get('spisok', ""),
                           "read":request.form.get('read', ""),
                           "write":request.form.get('write', ""),
                           "delete":request.form.get('delete', ""),
                       }})
    return redirect(f"/adminApp?id={appID}")

@app.route("/adminDelete",methods=['POST'])
def adminDelete():
    print(request.form)
    if request.form.get("adminPassword","") != ADMIN_PASSWORD:
        return redirect("/saveError")
    appID=request.form.get('appID', "")
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    db.apps.delete_one({"_id":ObjectId(appID)})
    return redirect("/admin")

@app.route('/secretKey')
def secretKey():
    linkID=request.args.get('link_id',"Ошибка")
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    linkObject=db.links.find_one({"_id":ObjectId(linkID)})
    appObject=db.apps.find_one({"_id":ObjectId(linkObject["appID"])})
    db.links.delete_one({"_id":ObjectId(linkID)})
    return render_template("secretKey.html",appObject=appObject)
    #return "lol"
    #return render_template("created.html",appID=appID)

app.run(host=PORTAL_HOST, port=PORTAL_PORT, threaded=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:49051/ (Press CTRL+C to quit)
127.0.0.1 - - [01/May/2021 12:50:22] "GET /create HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2021 12:50:32] "POST /createSave HTTP/1.1" 302 -
127.0.0.1 - - [01/May/2021 12:50:32] "GET /secretKey?link_id=608d246891a8896657a8fba0 HTTP/1.1" 200 -


In [1]:
PORTAL_PORT=49051
PORTAL_HOST='0.0.0.0'
DATABASE_PORT=27017

from bson.objectid import ObjectId
import random
import string
import hmac
import hashlib
from hashlib import sha256
from flask import Flask
from flask import request,redirect
from flask import Response
from flask import render_template
from os import listdir
from os.path import isfile, join
import os
from pymongo import MongoClient
import os.path

if os.path.isfile('.ADMIN_PASSWORD'):
    with open('.ADMIN_PASSWORD','r') as f:
        ADMIN_PASSWORD = f.read()
else:
    with open('.ADMIN_PASSWORD','w') as f:
        ADMIN_PASSWORD = "ADMIN_PASSWORD"
        f.write(ADMIN_PASSWORD)
        
appID="608d00f06f8e935b0408f24f"
client = MongoClient(port=DATABASE_PORT)
db=client.srp
db.apps.update_one({"_id":ObjectId(appID)},
                   {'$set': 
                   {
                       "spisok":"on",
                   }})

In [3]:
import random
import string
import hmac
import hashlib
from hashlib import sha256
from flask import Flask
from flask import request,redirect
from flask import Response
from flask import render_template
from os import listdir
from os.path import isfile, join
import os
from pymongo import MongoClient
sha256(("LsYLrhKJ1e"+"supersupersuper").encode('utf-8')).hexdigest()=="3c68d71c26d9b75689b56ed6d1a8820c53bbaaa91da35401b7c8d2188ebc1525"

True

In [ ]:
from base64 import b64encode
from collections import OrderedDict
from hashlib import sha256
from hmac import HMAC
from urllib.parse import urlparse, parse_qsl, urlencode


In [12]:
PORTAL_PORT=49051
PORTAL_HOST='0.0.0.0'
DATABASE_PORT=27017

from bson.objectid import ObjectId
import random
import string
import hmac
import hashlib
from hashlib import sha256
from flask import Flask
from flask import request,redirect
from flask import Response
from flask import render_template
from os import listdir
from os.path import isfile, join
import os
from pymongo import MongoClient

client = MongoClient(port=DATABASE_PORT)
db=client.srp
m=db.apps.find_one({"_id":ObjectId("608c0e27e9e834577d80daa7")})
m

{'_id': ObjectId('608c0e27e9e834577d80daa7'),
 'name': 'Блокнот',
 'desc': 'Для записей',
 'spisok': 'off',
 'read': 'zapros',
 'write': 'zapros',
 'delete': 'off',
 'salt': 'uCaMoI4bKC',
 'passwordHash': 'a812082d47d7d6814dea0394eb51d9a7128c0222b26f330fb891520320a60b8e',
 'secretKey': '8b34595526a3b82a32252c9cf89f89424880d12e717128b93df0b4efc735fcff'}

In [8]:
print(m)


In [3]:
a=[]
a.append(1)
a.append(2)
a

[1, 2]

In [4]:
with open('.ADMIN_PASSWORD','r') as f:
    ADMIN_PASSWORD = f.read()
ADMIN_PASSWORD

'lol'